In [23]:
import pandas
# import urllib.request
# requests works better...I'm not sure why
#but my encoding problem is gone.
import requests
import json
import geojson
import folium

In [26]:
#GDELT GeoJson Endpoint
url = "https://api.gdeltproject.org/api/v1/gkg_geojson"

#Issue request to URL
request = requests.get(url)
print(type(request))
data = request.json()
# with urllib.request.urlopen(url) as response:
#     html = response.read()
    


<class 'requests.models.Response'>


In [25]:
#get data
data = json.loads(request)
# print(type(data))

TypeError: the JSON object must be str, bytes or bytearray, not 'Response'